<div class="alert alert-block alert-success">
<center><b>AIM404 : Contextual Bandits with Amazon SageMaker RL - Inference Loop</b></center>
</div>

<a id='configuration'></a>
## Configuration

In [ ]:
import boto3, time, os, urllib.request, numpy, json
import time
from time import gmtime, strftime
import re
from sagemaker import get_execution_role
from sagemaker.predictor import RealTimePredictor
import sagemaker
import numpy as np
import sys

<div class="alert alert-block alert-danger"">
<b>IMPORTANT : </b>Setup the endpoint_name with the current sagemaker endpoint.
</div>

<div class="alert alert-block alert-info">
§ Open the <a href="https://us-east-1.console.aws.amazon.com/sagemaker/home?force&region=us-east-1#/endpoints">Sagemaker Endpoints console</a> to retrieve endpoint name
</div>

Here we will use the [RealTimePredictor](https://sagemaker.readthedocs.io/en/stable/predictors.html) from Sagemaker Python SDK

In [ ]:
endpoint_name = 'YOUR_SAGEMAKER_ENDPOINT'#Your sagemaker endpoint here
sess = sagemaker.Session()
realtime_predictor = RealTimePredictor(endpoint_name, 
                                       serializer=sagemaker.predictor.json_serializer,
                                       deserializer=sagemaker.predictor.json_deserializer,
                                       sagemaker_session=sess)

## Generate Context for inference loop

In [ ]:
file_name = 'sim_app/shuttle.trn'
num_actions = 7
data_size = 43483
  
def remove_underrepresented_classes(features, labels, thresh=0.0005):
    """Removes classes when number of datapoints fraction is below a threshold."""
    total_count = labels.shape[0]
    unique, counts = np.unique(labels, return_counts=True)
    ratios = counts.astype('float') / total_count
    vals_and_ratios = dict(zip(unique, ratios))
    print('Unique classes and their ratio of total: %s' % vals_and_ratios)
    keep = [vals_and_ratios[v] >= thresh for v in labels]
    return features[keep], labels[np.array(keep)]

with open(file_name, 'r') as f:
    data = np.loadtxt(f)

    # Shuffle data
    np.random.shuffle(data)

    # Last column is label, rest are features
    contexts = data[:, :-1]
    labels = data[:, -1].astype(int) - 1  # convert to 0 based index
    context, labels = remove_underrepresented_classes(contexts, labels)

## Inference Loop

Let's generate a loop over the sagemaker endpoint for inference

In [ ]:
for i in range(data_size):
    time.sleep(1)
    payload = {}
    payload['request_type'] = "observation"
    payload['observation'] = context[i]
    realtime_predictor.predict(payload)

<div class="alert alert-block alert-danger"">
Now go back to the previous notebook
</div>